# Starting Spark Session

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q !wget -q http://apache.osuosl.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop3.2.tgz
!tar xf spark-3.0.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
!pip install -U pyarrow

     |████████████████████████████████| 20.7MB 3.3MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop3.2"

In [5]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml.feature import StopWordsRemover

In [6]:
if __name__ == "__main__":
	
	# create Spark context with necessary configuration
	sc = SparkContext("local","PySpark Word Count Exmaple")

# Extracting names of all the news categories.

In [7]:
category_set1=list()
import re
with open('/content/drive/My Drive/Colab Notebooks/urls.txt', 'r') as f: 
    for line in f:
      obj2 = re.findall('://[\w\-\.]+/[0-9]+/[0-9]+/[0-9]+/([\w\-]+)',  
                  line) 
      str1 = ''.join(str(e) for e in obj2)
      #print(str1)
      category_set1.append(str1)
category_set2 = set(category_set1)
category_list = list(category_set2)
for category_set3 in category_list:
  if category_set3 == '':
    category_list.remove(category_set3)

In [8]:
check=[]
for i in category_list:
  category='/'+i+'/'
  check.append(category)

# Spliting the entire nytimes file at URL

In [9]:
noe = list()
with open('/content/drive/My Drive/Colab Notebooks/nytimes.txt', 'r') as f: 
    x=f.read()
x=x.lower()
noe=x.split('url:')

# Forming the list of top 10 words from the news articles of each category.

In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

#For storing final output
print_data = list()
#For storing all the articles of a particular category
category_data = list()
#For storing word and count of top 10 words
zz=list()
gg=list()
for ls in check:
  category_data.clear()
  print_data.clear()
  print_data.append("Category: "+ ls)
  for words in noe:
    if re.search(ls, words):  
      category_data.append(words) 
    else:
      pass
  file1 = open('/content/drive/My Drive/Colab Notebooks/temp_save.txt', 'w') 
  for ddd in category_data:
    file1.write(ddd+'\n')
  file1.close()


  ####    Punctuation removal
  with open('/content/drive/My Drive/Colab Notebooks/temp_save.txt', 'r') as f: 
    x=f.read()
  x=x.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  y=tokenizer.tokenize(x)
  filtered_sentence = (" ").join(y)

  file1 = open('/content/drive/My Drive/Colab Notebooks/temp_save.txt', 'w') 
  file1.write(filtered_sentence)
  file1.close()

  ####    WORDCOUNT   
  # Define a list of stop words or use default list
  remover = StopWordsRemover()
  stop_words = remover.getStopWords() 
  # read data from text file and split each line into words
  words = sc.textFile('/content/drive/My Drive/Colab Notebooks/temp_save.txt').flatMap(lambda line: line.split(" "))
  # Convert the words in lower case and remove stop words from stop_words
  splitRDD_no_stop = words.filter(lambda x: x.lower() not in stop_words)

  # Create a tuple of the word and 1 
  splitRDD_no_stop_words = splitRDD_no_stop.map(lambda w: (w, 1))

  # Count of the number of occurences of each word 
  wordCounts = splitRDD_no_stop_words.reduceByKey(lambda x, y: x + y)
  #wordCounts= words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
  wordCounts = wordCounts.map(lambda x:(x[1],x[0]))
  wordCounts=wordCounts.sortByKey(False)
  wordCounts = wordCounts.map(lambda x:(x[1],x[0]))
  xy=wordCounts.take(10)



  #Forming the list of top 10 words
  zz.clear()
  gg.clear()
  for xx in xy:
    zz.append(xx[1])
    gg.append(xx[0])
  #Storing the top 10 words
  for ee in range(10):
    print_data.append("Word:"+gg[ee]+"   Count:"+str(zz[ee]))
  


  #Forming the file
  file1 = open('/content/drive/My Drive/Colab Notebooks/out_task3.txt', 'a')
  for ddd in print_data:
    file1.write(ddd+'\n')
  file1.write('\n\n\n')
  file1.close()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
